# Part I. ETL Pipeline for Pre-Processing the Files

In [59]:
# Import Python packages 
import pandas as pd
import cassandra
import os
import glob
import csv

#### Creating list of filepaths to process original event csv data files

In [60]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

#Find all csv files in specified file path
file_path_list = [f for f in glob.glob(filepath + "/*.csv", recursive=True)]


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [61]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#print(len(full_data_rows_list))
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [62]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


### Creating a Cluster

In [78]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)
# To establish connection and begin executing queries, need a session
session = cluster.connect()

### Create Keyspace

In [79]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

### Set Keyspace

In [80]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Create DB and Query tables

#### First Query
*Artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4*

In Apache Cassandra primary keys are used to partion data in the table.  Our data will be partitioned on *"session_id"* and *"item_in_session"*, meaning that each row that corrospondes to these values will be unique in the table. A compound primary key is required here, because there may be multiple *item_in_sessions* per *session_id*.

We are only adding in the necessary columns to the database, therefore we will need our primary keys, followed by the features we wish to extract, *artist_name,* *song_name,* *song_lenth.*

In [81]:
##Artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS artist_song_length_by_session_item"
query = query + """(session_id int, 
                    item_in_session int,
                    artist_name text, 
                    song_name text, 
                    song_length float, 
                    PRIMARY KEY (session_id, item_in_session))"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [82]:
file = 'event_datafile_new.csv'

df = pd.read_csv(file)

#Loop through each row in dataframe and insert into DB
for i, row in df.iterrows():
    query = """INSERT INTO artist_song_length_by_session_item (
                session_id, 
                item_in_session,
                artist_name, 
                song_name, 
                song_length)
                VALUES (%s, %s, %s, %s, %s)"""
    
    session.execute(query, (row['sessionId'], row['itemInSession'], row['artist'], row['song'], row['length']))


#### Verifying data from query

In [84]:
query = """SELECT artist_name, 
                  song_name, 
                  song_length 
                  FROM artist_song_length_by_session_item 
                  WHERE session_id = 338 AND item_in_session = 4"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
    
for row in rows:
    print ("Artist Name: {}, Song Title: {}, Song Length: {}".format(row.artist_name, 
                                                                     round(row.song_length,0), 
                                                                     row.song_name))

Artist Name: Faithless, Song Title: 495.0, Song Length: Music Matters (Mark Knight Dub)


#### Second Query
*Name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182*

- Data has been paritioned on *"user_id"*, *"session_id"* and finally *"item_in_session"*
- Only the necessary features have been included in the creation of the table columns.
- A Clustering order has been specified on the *session_id* and *item_in_session*, meaning first of all items in that current partition will be sorted by *session_id* and secondly by the *item_in_session*.

In [85]:
## Name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

#First we create the database to model the query
query = """CREATE TABLE IF NOT EXISTS artist_song_user_by_user_id_session_id (
            user_id int,
            session_id int, 
            item_in_session int,
            artist_name text, 
            song_name text, 
            user_fname text, 
            user_lname text, 
            PRIMARY KEY (user_id, session_id, item_in_session)
            ) 
            WITH CLUSTERING ORDER BY (session_id ASC, item_in_session ASC)"""
                    
try: 
    session.execute(query)
except Exception as e:
    print (e)


In [86]:
#Next we open the CSV file up and insert in new database
file = 'event_datafile_new.csv'
df = pd.read_csv(file)

for i, row in df.iterrows():
    query = """INSERT INTO artist_song_user_by_user_id_session_id (
                user_id,
                session_id, 
                item_in_session, 
                artist_name, 
                song_name, 
                user_fname, 
                user_lname 
                ) 
                VALUES (%s,%s,%s,%s,%s,%s,%s)"""
    
    session.execute(query, (row['userId'], row['sessionId'], row['itemInSession'], row['artist'], row['song'], row['firstName'], row['lastName']))

In [87]:
#Next query the DB
query = """SELECT session_id, 
                  item_in_session,
                  artist_name,
                  song_name,
                  user_fname, 
                  user_lname
                  FROM artist_song_user_by_user_id_session_id 
                  WHERE user_id = 10 AND session_id = 182"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print("SESSION_ID: {}, SESSION_ITEM: {}, USER: {}, ARTIST: {}, SONG: {}".format(row.session_id, 
                                                                                    row.item_in_session,
                                                                                    row.user_fname+" "+row.user_lname, 
                                                                                    row.artist_name, 
                                                                                    row.song_name))

SESSION_ID: 182, SESSION_ITEM: 0, USER: Sylvie Cruz, ARTIST: Down To The Bone, SONG: Keep On Keepin' On
SESSION_ID: 182, SESSION_ITEM: 1, USER: Sylvie Cruz, ARTIST: Three Drives, SONG: Greece 2000
SESSION_ID: 182, SESSION_ITEM: 2, USER: Sylvie Cruz, ARTIST: Sebastien Tellier, SONG: Kilometer
SESSION_ID: 182, SESSION_ITEM: 3, USER: Sylvie Cruz, ARTIST: Lonnie Gordon, SONG: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


### Query 3
*Every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'*

- Data has been paritioned on *"user_id"*, *"session_id"* and finally *"item_in_session"*
- Only the necessary features have been included in the creation of the table columns.
- In order to avoid an "Allow Filtering" error, data is partitioned firstly on *song_id* (as this is specified in the where clause) then secondly on *user_id* 

In [88]:
#Every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#First create the query
query = """CREATE TABLE IF NOT EXISTS user_by_song (
            song text, 
            user_id int, 
            user_fname text, 
            user_lname text,
            PRIMARY KEY(song, user_id))"""

try:
    session.execute(query)
except Exception as e:
    print (e)

In [73]:
# Using dataframe insert into DB
file = 'event_datafile_new.csv'
df = pd.read_csv(file)

for i, row in df.iterrows():
    query = """INSERT INTO user_by_song (
                song, 
                user_id,
                user_fname, 
                user_lname) 
                VALUES (%s, %s, %s, %s)"""
    
    session.execute(query, ( row['song'], row['userId'], row['firstName'], row['lastName']))
    

In [74]:
#Select rows based upon query
query = "SELECT user_fname, user_lname from user_by_song WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print (e)

for row in rows:
    print("User: {}".format(row.user_fname + " " + row.user_lname))
    

User: Jacqueline Lynch
User: Tegan Levine
User: Sara Johnson


### Drop the tables before closing out the sessions

In [75]:
#Drop each of the tables
tables = ['artist_song_length_by_session_item', 'artist_song_user_by_user_id_session_id', 'user_by_song']
query = "drop table if exists {}"
for table in tables:
    try:
        rows = session.execute(query.format(table))
    except Exception as e:
        print(e)    

### Close the session and cluster connection¶

In [76]:
session.shutdown()
cluster.shutdown()